In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet1.zip', 'r') as zip_ref:
        zip_ref.extractall('./DataSet1')
except:
    print("Using Local Machine")

Using Local Machine


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15598, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 15598 (delta 98), reused 119 (delta 57), pack-reused 15393
Receiving objects: 100% (15598/15598), 14.64 MiB | 21.26 MiB/s, done.


In [ ]:
# Include all packages
import os
import gc
import cv2
import shutil
import numpy as np
import pandas as pd
from time import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from yolov5.models.yolo import Model
from sklearn.model_selection import train_test_split

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from copy import deepcopy


In [ ]:
def CannyEdge(capturedImage):
    grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
    constrastKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5) )
    topHat = cv2.morphologyEx(grayScale, cv2.MORPH_TOPHAT, constrastKernel)
    blackHat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, constrastKernel)
    grayScale = grayScale + topHat - blackHat
    bilateralFilter = cv2.bilateralFilter(grayScale, 11, 17, 17)
    imageMedian = np.median(capturedImage)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))
    cannyEdgeImage = cv2.Canny(bilateralFilter, lowerThreshold, upperThreshold)
    cannyEdgeImage = cv2.bitwise_not(cannyEdgeImage)
    cannyEdgeImage = cv2.cvtColor(cannyEdgeImage, cv2.COLOR_GRAY2BGR)
    return cannyEdgeImage

In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [uniqueSigns.index(row[5]), row[1], row[2], row[3], row[4]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = [[] for i in range(47)]
    newAnnotations = [[] for i in range(47)]
    for i, image in enumerate(images):
        [classIndex, x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
         # xCenter = ((x1New + x2New) / 2) / resize[0]
        # yCenter = ((y1New + y2New) / 2) /  resize[1]
        # width = (x2New - x1New) /  resize[0]
        # height = (y2New - y1New) /  resize[1]
        # newAnnotations[classIndex].append(
        #     [xCenter, yCenter, width, height])

        # resizedImage = CannyEdge(resizedImage)
        resizedImages[classIndex].append(resizedImage)
        newAnnotations[classIndex].append(
            [x1New, y1New, x2New, y2New])
    # AllX_train=[]
    # AllX_val = []
    # Ally_train = [] 
    # Ally_val = []
    # for i in range(47):
    #     X_train, X_val, y_train, y_val = train_test_split(
    #         resizedImages[i], newAnnotations[i], test_size=0.1, random_state=42)
    #     AllX_train.append(X_train)
    #     AllX_val.append(X_val)
    #     Ally_train.append(y_train) 
    #     Ally_val.append(y_val)

    # return AllX_train, AllX_val, Ally_train, Ally_val
    return resizedImages, newAnnotations


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    # model.load_state_dict(torch.load("yolov5{}.pt".format(version))["model"].state_dict(), strict=False)

    return model


In [ ]:
def DetectImage(model, inputs, device, conf_thres=0.2, iou_thres=0.5):
    model.eval()

    inputs = torch.tensor(inputs, dtype=torch.float32)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.to(device)
    conf_thres = torch.tensor(conf_thres)
    with torch.no_grad():
        output = model(inputs)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        # output = torchvision.ops.nms(output, conf_thres, iou_thres)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        output = output[0]
        box_coordinates = output[..., :4].view(-1, 4)
        confidence_scores = output[..., 4].view(-1)
        nms_indices = torchvision.ops.nms(box_coordinates, confidence_scores, iou_thres)
        output = output.view(-1, output.shape[-1])[nms_indices]
    # Remove the batch dimension
    output = output.squeeze(0)
    return output


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 100
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
images, annotations, numClasses = LoadDataSet("./DataSet1")
# numClasses = 1

In [ ]:
X_train, y_train = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
X_val = [[] for i in range(47)]
y_val = [[] for i in range(47)]
del images
del annotations
gc.collect()

In [ ]:
# trainDataLoader, valDataLoader = CreateDataLoaders(
#     X_train, X_val, y_train, y_val, batchSize)
# del X_train
# del y_train
# del X_val
# del y_val
gc.collect()

In [ ]:
yolov5Model = CreateYolov5Model(numClasses,yolo5Version)
yolov5Model = yolov5Model.to(device)


In [ ]:
savedModels = [ modelName for modelName in os.listdir('/content/drive/MyDrive/DL Project/Trained Models/') if ('yolov5SingleModelv2' in modelName)]
savedModels.sort()
print(savedModels[-1])
yolov5Model.load_state_dict(torch.load('/content/drive/MyDrive/DL Project/Trained Models/'+savedModels[-1], map_location=torch.device(device)))


In [ ]:
import random
randInt = random.randint(0,len(X_val))

image = X_val[randInt]
image1 = deepcopy(image)
# try:
#     from google.colab.patches import cv2_imshow
#     cv2_imshow(image)
# except:
#     print("using Local")
#     cv2.imshow("Input Image", image)

predictions = DetectImage(yolov5Model, image, device)
[a1,b1,a2,b2] = y_val[randInt]
bBoxs = [[a1,b1,a2,b2]]
machingbBoxes = []
albBoxs =[]
i=0
for pred in predictions:
    
    i+=1
    # print(predictions.shape)
    # for p in pred:
    #   print(int(p))
    # break
    x1, y1, x2, y2, m1,m2 = pred[:6]
    # x2 = x1+x2
    # y2 = y1+y2
    # albBoxs.append( [x1, y1, x1+x2, y2])
    m1,m2, x1, y1, x2, y2= int(m1), int(m2),int(x1), int(y1), int(x2), int(y2)
    if(a1 == x1 or a2 == x2 or b1 == y1 or b2 == y2 ):
      if(((x1-x2) >= 17 and (x1-x2) <= 32) and ((y1-y2) >= 31 and (y1-y2)<= 56) ):
        machingbBoxes.append([x1, y1, x2,y2])
      
    if(((x1-x2) >= 17 and (x1-x2) <= 32) and ((y1-y2) >= 31 and (y1-y2)<= 56) ):
        bBoxs.append([x1, y1, x2, y2])
    # if(((x2) >= 17 and (x2) <= 44) and ((y2) >= 31 and (y2)<= 56)):
    #     bBoxs1.append([x1, y1, x1+x2, y1+y2])
    # x3 = abs((x1-x2)//2)
    # x4 = (x1+x2)//2
    # y3 = abs((y1-y2)//2)
    # y4 = (y1+y2)//2
    # if(((x4-x3) >= 17 and (x4-x3) <= 44) and ((y4-y3) >= 31 and (y4-y3)<= 76) ):
    #     bBoxs2.append([x3, y3, x4, y4])

print("No. Objects detected:" ,len(bBoxs) )
print("No. Objects detected:" ,len(machingbBoxes) )
# print("No. Objects detected:" ,len(bBoxs1) )
# print("No. Objects detected:" ,len(bBoxs2) )
# [x1, y1, x2, y2] = bBoxs[0]
# cv2.rectangle(image1, (x1, y1), (x2, y2), (0,255,0), 2)

cv2.rectangle(image, (a1, b1), (a2, b2), (0,255,0), 2)

for bBox in bBoxs[1:]:
    # print(bBox)
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)
try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image)
except:
    print("using Local")
    cv2.imshow("Input Image", image)



In [ ]:
cv2.rectangle(image, (a1, b1), (a2, b2), (0,255,0), 2)

for bBox in bBoxs[1:]:
    # print(bBox)
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)
try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image)
except:
    print("using Local")
    cv2.imshow("Input Image", image)

In [ ]:
cv2.rectangle(image1, (a1, b1), (a2, b2), (0,255,0), 2)

for bBox in machingbBoxes:
    [x1, y1, x2, y2] = bBox
    cv2.rectangle(image1, (x1, y1), (x2, y2), (0,0,255), 2)
try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image1)
except:
    print("using Local")
    cv2.imshow("Input Image", image1)